# 🚀 NASA GCN Data Pipeline

## Visão Geral
Pipeline **Delta Live Tables (DLT)** para ingestão de eventos astronômicos da NASA Gamma-ray Coordinates Network (GCN).

### O que é o NASA GCN?
O [GCN (Gamma-ray Coordinates Network)](https://gcn.nasa.gov/) é um sistema de alerta em tempo real que distribui informações sobre:
- 🌟 **Gamma-ray Bursts (GRBs)** - Explosões de raios gama
- 🌊 **Ondas Gravitacionais (IGWN)** - Detecções do LIGO/Virgo
- ☄️ **Eventos de Neutrinos** - IceCube e outros detectores
- 🔭 **Transientes Astronômicos** - Supernovas, kilonovas, etc.

---

## Arquitetura Medallion (Bronze → Silver → Gold)

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│     BRONZE      │    │     SILVER      │    │      GOLD       │
│   (Raw Data)    │───▶│  (Parsed Data)  │───▶│  (Aggregated)   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
     gcn_raw           gcn_classic_text            (futuro)
                       gcn_classic_voevent
                       gcn_classic_binary
                       gcn_notices
                       gcn_circulars
                       igwn_gwalert
                       gcn_heartbeat
```

### Camadas:
| Camada | Propósito | Tabelas |
|--------|-----------|----------|
| **Bronze** | Dados brutos exatamente como chegaram do Kafka | `gcn_raw` |
| **Silver** | Dados filtrados por tipo, decodificados, com schema definido | 7 tabelas por categoria |
| **Gold** | Agregações, métricas, dashboards (implementação futura) | - |

## Configuração e Importações

In [ ]:
# ==============================================================================
# IMPORTS E CONFIGURAÇÃO
# ==============================================================================

import dlt
import sys
import struct
from datetime import datetime, timedelta

sys.path.append(spark.conf.get("bundle.sourcePath", "."))

from pyspark.sql.functions import col, decode, split, current_timestamp, current_date, udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

from nasa_gcn.config import get_kafka_options

## Parser de Pacotes Binários GCN (160 bytes)

Mapeamento completo baseado em: https://gcn.gsfc.nasa.gov/sock_pkt_def_doc.html

In [ ]:
# ==============================================================================
# MAPEAMENTO COMPLETO DE TIPOS DE PACOTES GCN
# ==============================================================================

PACKET_TYPE_NAMES = {
    # Básicos/Sistema
    1: "BATSE_ORIGINAL", 2: "TEST", 3: "IMALIVE", 4: "KILL",
    # BATSE/ALEXIS (legado)
    11: "BATSE_MAXBC", 21: "BRADFORD_TEST", 22: "BATSE_FINAL", 24: "BATSE_LOCBURST", 25: "ALEXIS",
    # RXTE
    26: "RXTE_PCA_ALERT", 27: "RXTE_PCA", 28: "RXTE_ASM_ALERT", 29: "RXTE_ASM", 30: "COMPTEL",
    # IPN
    31: "IPN_RAW", 32: "IPN_SEGMENT", 33: "SAX_WFC_ALERT", 34: "SAX_WFC",
    35: "SAX_NFI_ALERT", 36: "SAX_NFI", 37: "RXTE_ASM_XTRANS", 38: "SPARE_TESTING", 39: "IPN_POSITION",
    # HETE
    40: "HETE_S/C_ALERT", 41: "HETE_S/C_UPDATE", 42: "HETE_S/C_LAST",
    43: "HETE_GNDANA", 44: "HETE_TEST", 45: "GRB_COUNTERPART",
    46: "SWIFT_TOO_FOM_OBSERVE", 47: "SWIFT_TOO_SC_SLEW", 48: "DOW_TOD_TEST",
    # INTEGRAL
    51: "INTEGRAL_POINTDIR", 52: "INTEGRAL_SPIACS", 53: "INTEGRAL_WAKEUP",
    54: "INTEGRAL_REFINED", 55: "INTEGRAL_OFFLINE", 56: "INTEGRAL_WEAK",
    57: "AAVSO", 58: "MILAGRO", 59: "KONUS_LIGHTCURVE",
    # SWIFT (60-99, 103, 133, 140, 141)
    60: "SWIFT_BAT_GRB_ALERT", 61: "SWIFT_BAT_GRB_POSITION",
    62: "SWIFT_BAT_GRB_NACK", 63: "SWIFT_BAT_GRB_LC",
    64: "SWIFT_BAT_SCALED_MAP", 65: "SWIFT_FOM_OBSERVE", 66: "SWIFT_SC_SLEW",
    67: "SWIFT_XRT_POSITION", 68: "SWIFT_XRT_SPECTRUM", 69: "SWIFT_XRT_IMAGE",
    70: "SWIFT_XRT_LIGHTCURVE", 71: "SWIFT_XRT_NACK_POSITION",
    72: "SWIFT_UVOT_IMAGE", 73: "SWIFT_UVOT_SRC_LIST",
    76: "SWIFT_BAT_GRB_PROC_LC", 77: "SWIFT_XRT_PROC_SPECTRUM",
    78: "SWIFT_XRT_PROC_IMAGE", 79: "SWIFT_UVOT_PROC_IMAGE", 80: "SWIFT_UVOT_PROC_SRC_LIST",
    81: "SWIFT_UVOT_POSITION", 82: "SWIFT_BAT_GRB_POS_TEST", 83: "SWIFT_POINTDIR",
    84: "SWIFT_BAT_TRANS", 85: "SWIFT_XRT_THRESHPIX", 86: "SWIFT_XRT_THRESHPIX_PROC",
    87: "SWIFT_XRT_SPER", 88: "SWIFT_XRT_SPER_PROC", 89: "SWIFT_UVOT_NACK_POSITION",
    97: "SWIFT_BAT_QUICKLOOK_POSITION", 98: "SWIFT_BAT_SUBTHRESHOLD_POSITION",
    99: "SWIFT_BAT_SLEW_GRB_POSITION", 103: "SWIFT_ACTUAL_POINTDIR",
    133: "SWIFT_BAT_MONITOR", 140: "SWIFT_BAT_SUB_SUB_THRESH_POS", 141: "SWIFT_BAT_KNOWN_SRC_POS",
    # SuperAGILE/AGILE
    100: "SUPERAGILE_GRB_WAKEUP", 101: "SUPERAGILE_GRB_GROUND", 102: "SUPERAGILE_GRB_REFINED",
    105: "AGILE_MCAL_ALERT", 107: "AGILE_POINTDIR", 109: "SUPERAGILE_GRB_POS_TEST",
    # FERMI GBM
    110: "FERMI_GBM_ALERT", 111: "FERMI_GBM_FLT_POS", 112: "FERMI_GBM_GND_POS",
    114: "FERMI_GBM_GND_INTERNAL", 115: "FERMI_GBM_FINAL_POS",
    116: "FERMI_GBM_ALERT_INTERNAL", 117: "FERMI_GBM_FLT_INTERNAL", 119: "FERMI_GBM_POS_TEST",
    131: "FERMI_GBM_SUBTHRESHOLD",
    # FERMI LAT
    120: "FERMI_LAT_GRB_POS_INI", 121: "FERMI_LAT_GRB_POS_UPD", 122: "FERMI_LAT_GRB_POS_DIAG",
    123: "FERMI_LAT_TRANS", 124: "FERMI_LAT_GRB_POS_TEST", 125: "FERMI_LAT_MONITOR",
    126: "FERMI_SC_SLEW", 127: "FERMI_LAT_GND", 128: "FERMI_LAT_OFFLINE", 129: "FERMI_POINTDIR",
    144: "FERMI_SC_SLEW_INTERNAL", 146: "FERMI_GBM_FIN_POS_INTERNAL",
    # MISC
    130: "SIMBAD_NED_SEARCH_RESULTS", 134: "MAXI_UNKNOWN_SOURCE", 135: "MAXI_KNOWN_SOURCE",
    136: "MAXI_TEST", 137: "OGLE", 139: "MOA", 145: "COINCIDENCE", 148: "SUZAKU_LIGHTCURVE", 149: "SNEWS",
    # LVC (LIGO/Virgo/KAGRA)
    150: "LVC_PRELIMINARY", 151: "LVC_INITIAL", 152: "LVC_UPDATE",
    153: "LVC_TEST", 154: "LVC_COUNTERPART", 163: "LVC_EARLY_WARNING", 164: "LVC_RETRACTION",
    # AMON/ICECUBE/CALET
    157: "AMON_ICECUBE_COINC", 158: "AMON_ICECUBE_HESE", 159: "AMON_ICECUBE_TEST",
    160: "CALET_GBM_FLT_LC", 161: "CALET_GBM_GND_LC", 166: "AMON_ICECUBE_CLUSTER",
    168: "GWHEN_COINC", 169: "AMON_ICECUBE_EHE", 170: "AMON_ANTARES_FERMILAT_COINC",
    171: "HAWC_BURST_MONITOR", 172: "AMON_NU_EM_COINC",
    173: "ICECUBE_ASTROTRACK_GOLD", 174: "ICECUBE_ASTROTRACK_BRONZE",
    175: "SK_SUPERNOVA", 176: "AMON_ICECUBE_CASCADE",
    # GECAM
    188: "GECAM_FLT", 189: "GECAM_GND",
}

TJD_EPOCH = datetime(1968, 5, 24, 0, 0, 0)

PARSED_BINARY_SCHEMA = """
    pkt_type INT, pkt_type_name STRING, pkt_sernum INT, trig_num INT,
    burst_tjd INT, burst_sod_centi INT, burst_datetime STRING,
    burst_ra_deg DOUBLE, burst_dec_deg DOUBLE, burst_error_deg DOUBLE,
    trigger_id INT, misc INT, parse_error STRING
"""

def parse_gcn_binary_packet(binary_data):
    result = {"pkt_type": None, "pkt_type_name": None, "pkt_sernum": None,
              "trig_num": None, "burst_tjd": None, "burst_sod_centi": None,
              "burst_datetime": None, "burst_ra_deg": None, "burst_dec_deg": None,
              "burst_error_deg": None, "trigger_id": None, "misc": None, "parse_error": None}
    
    if binary_data is None:
        result["parse_error"] = "binary_data is None"
        return result
    if len(binary_data) != 160:
        result["parse_error"] = f"Invalid packet size: {len(binary_data)} bytes"
        return result
    
    try:
        import struct
        from datetime import datetime, timedelta
        longs = struct.unpack('>40i', binary_data)
        
        pkt_type = longs[0]
        result["pkt_type"] = pkt_type
        result["pkt_type_name"] = PACKET_TYPE_NAMES.get(pkt_type, f"UNKNOWN_{pkt_type}")
        result["pkt_sernum"] = longs[1]
        result["trig_num"] = longs[4] if longs[4] > 0 else None
        
        burst_tjd, burst_sod = longs[5], longs[6]
        result["burst_tjd"], result["burst_sod_centi"] = burst_tjd, burst_sod
        
        if burst_tjd > 0 and burst_sod >= 0:
            try:
                TJD_EPOCH = datetime(1968, 5, 24, 0, 0, 0)
                burst_dt = TJD_EPOCH + timedelta(days=burst_tjd, seconds=burst_sod / 100.0)
                result["burst_datetime"] = burst_dt.isoformat()
            except: pass
        
        burst_ra, burst_dec, burst_error = longs[7], longs[8], longs[11]
        scale = 10000 if (burst_ra > 36000 or burst_ra < 0 or abs(burst_dec) > 9000) else 100
        
        ra_deg, dec_deg = burst_ra / scale, burst_dec / scale
        result["burst_ra_deg"] = ra_deg if 0 <= ra_deg < 360 else None
        result["burst_dec_deg"] = dec_deg if -90 <= dec_deg <= 90 else None
        result["burst_error_deg"] = abs(burst_error) / scale
        result["trigger_id"], result["misc"] = longs[18], longs[19]
    except Exception as e:
        result["parse_error"] = str(e)
    return result

parse_binary_udf = udf(parse_gcn_binary_packet, PARSED_BINARY_SCHEMA)

---

## 🥉 Bronze Layer: Ingestão Raw

In [ ]:
@dlt.table(name="gcn_raw", comment="Raw NASA GCN Kafka messages - Bronze layer",
           table_properties={"quality": "bronze"})
def gcn_raw():
    kafka_options = get_kafka_options()
    return (spark.readStream.format("kafka").options(**kafka_options).load()
        .select(
            col("key").cast("string").alias("message_key"),
            col("value"), col("topic"), col("partition"), col("offset"),
            col("timestamp").alias("kafka_timestamp"),
            current_timestamp().alias("ingestion_timestamp")))

---

## 🥈 Silver Layer: Tabelas Multiplexadas por Categoria

In [ ]:
@dlt.table(name="gcn_classic_text", comment="Classic text format GCN alerts - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_text():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.text."))
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("message_text"),
            col("topic"), split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_classic_voevent", comment="Classic VoEvent XML format GCN alerts - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_voevent():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.voevent."))
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("voevent_xml"),
            col("topic"), split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_classic_binary", comment="Classic binary format GCN alerts with parsed fields - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_binary():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.binary."))
        .withColumn("parsed", parse_binary_udf(col("value")))
        .select(
            col("message_key"),
            col("parsed.pkt_type"), col("parsed.pkt_type_name"), col("parsed.pkt_sernum"),
            col("parsed.trig_num"), col("parsed.burst_datetime"),
            col("parsed.burst_ra_deg"), col("parsed.burst_dec_deg"), col("parsed.burst_error_deg"),
            col("parsed.trigger_id"), col("parsed.parse_error"),
            col("topic"), split(col("topic"), r"\.").getItem(3).alias("event_type"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date"),
            col("value").alias("raw_binary")))

In [ ]:
@dlt.table(name="gcn_notices", comment="New JSON format GCN notices - Silver layer",
           table_properties={"quality": "silver"})
def gcn_notices():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.notices."))
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("notice_json"),
            col("topic"), split(col("topic"), r"\.").getItem(2).alias("mission"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_circulars", comment="GCN Circulars - astronomer reports - Silver layer",
           table_properties={"quality": "silver"})
def gcn_circulars():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.circulars")
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("circular_json"),
            col("topic"), col("kafka_timestamp"), col("ingestion_timestamp"),
            col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="igwn_gwalert", comment="IGWN Gravitational Wave Alerts - Silver layer",
           table_properties={"quality": "silver"})
def igwn_gwalert():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "igwn.gwalert")
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("gwalert_json"),
            col("topic"), col("kafka_timestamp"), col("ingestion_timestamp"),
            col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_heartbeat", comment="GCN Heartbeat messages for monitoring - Silver layer",
           table_properties={"quality": "silver"})
def gcn_heartbeat():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.heartbeat")
        .select(
            col("message_key"), decode(col("value"), "UTF-8").alias("heartbeat_json"),
            col("topic"), col("kafka_timestamp"), col("ingestion_timestamp"),
            col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))